<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Домашнее-Задание-#2" data-toc-modified-id="Домашнее-Задание-#2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Домашнее Задание #2</a></span></li></ul></div>

# Домашнее Задание #2
+ Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
+ Cохранить ее в pandas dataframe
+ Cохранить pandas dataframe в excel
+ Cохранить pandas dataframe в pickle
+ Cохранить pandas dataframe в БД


In [18]:
import requests,math
from tqdm.auto import tqdm

In [31]:
offset_ = 1 # размер шага
limit_ = 1000 # количество объектов в выдаче
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'

In [32]:
# Отправляем GET запрос в api.hh.ru
res = requests.get(url) 
# Преобразовываем ответ api из JSON в словарь dict()
objects_data = res.json()
#количество объектов всего
objects_data['data']['total']

11693

In [35]:
num_pages=math.ceil(objects_data['data']['total']/limit_)
num_pages

12

In [64]:
# Запускаем цикл для сбора всех идентификаторов объектов со всех страниц
all_object_ids= []
for offset_ in tqdm(range(1500)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json() 
    objids = [x.get('objId') for x in objects_data.get('data').get('list')]
    all_object_ids.extend(objids)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [73]:
all_object_ids=set(all_object_ids)
len(all_object_ids)

2499

In [74]:
# Собираем массив с полными описаниями строящихся объектов по идентиф икаторам
full_object = []
for obj_id in tqdm(all_object_ids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}'
    res = requests.get(url)
    objects = res.json()
    full_object.append(objects)

  0%|          | 0/2499 [00:00<?, ?it/s]

In [75]:
# Создаем таблицу DataFrame из всех полученных данных

In [76]:
import pandas as pd

In [77]:
df = pd.json_normalize(full_object)

In [78]:
df.head()

,errcode,data.id,data.hobjId,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,...,data.metro.isWalk,data.metro.colors,data.greenAreaIndexValue,data.airQualityIndexValue,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.rnvFlg,data.rnvDTO
0,0,49157,129796,656916,14800,АТЛАНТИК,АТЛАНТИК,АТЛАНТИК,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,49176,129816,602084,14246,СЗ БАКУНИНСКИЙ,СЗ БАКУНИНСКИЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК БАКУНИНСКИЙ,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,49177,129815,602084,14246,СЗ БАКУНИНСКИЙ,СЗ БАКУНИНСКИЙ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК БАКУНИНСКИЙ,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,8224,12514,1570,804,ЗОЛОТАЯ МИЛЯ,ЗОЛОТАЯ МИЛЯ,ЗОЛОТАЯ МИЛЯ,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,41000,121531,460846,1653,ГК ЖБИ-2,ГК ЖБИ-2,ГРУППА КОМПАНИЙ ЖБИ-2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df.shape

(2499, 113)

In [80]:
# Сохраняем в csv файл
df.to_csv('object.csv', index=False)
# Сохраняем в эксель-файл
df.to_excel('object.xlsx', index=False)
# Сохраняем в файл в формате pickle
df.to_pickle('object.pkl')